In [22]:
import pandas as pd

# #Flan-T5 (780M)
# df = pd.read_csv("SP-train-flan-t5-780M.csv")
# df.head()

#Roberta-l
df = pd.read_csv('robertal.csv')
df.head()

,id,question,answer,distractor1,distractor2,distractor(unsure),label,choice_list,choice_order,prediction,correct,group
0,SP-0,Mr. and Mrs. Mustard have six daughters and ea...,Each daughter shares the same brother.,Some daughters get married and have their own ...,Some brothers were not loved by family and mov...,None of above.,1,[Some daughters get married and have their own...,"[1, 0, 2, 3]",1,1,OG
1,SP-0_SR,The six daughters of Mr. and Mrs. Mustard each...,Each daughter shares the same brother.,Some daughters get married and have their own ...,Some brothers were not loved by family and mov...,None of above.,2,[Some brothers were not loved by family and mo...,"[2, 1, 0, 3]",2,1,OG
2,SP-0_CR,"A chess team has five players, and each player...",Each player shares the same coach.,Some coaches get a raise.,Some players are backups and not allowed to play.,None of above.,0,"[Each player shares the same coach., Some play...","[0, 2, 1, 3]",0,1,CR
3,SP-1,A woman shoots her husband. Then she holds him...,The woman was a photographer. She shot a pictu...,The woman gets arrested for murder after dinner.,The woman gets a new partner.,None of above.,2,[The woman gets arrested for murder after dinn...,"[1, 2, 0, 3]",2,1,OG
4,SP-1_SR,An individual shoots their spouse. She continu...,The woman was a photographer. She shot a pictu...,The woman gets arrested for murder after dinner.,The woman gets a new partner.,None of above.,1,[The woman gets arrested for murder after dinn...,"[1, 0, 2, 3]",1,1,OG


In [23]:
df['stem'] = df['id'].str.split("_").str[0]


,id,question,answer,distractor1,distractor2,distractor(unsure),label,choice_list,choice_order,prediction,correct,group,stem
0,SP-0,Mr. and Mrs. Mustard have six daughters and ea...,Each daughter shares the same brother.,Some daughters get married and have their own ...,Some brothers were not loved by family and mov...,None of above.,1,[Some daughters get married and have their own...,"[1, 0, 2, 3]",1,1,OG,SP-0
1,SP-0_SR,The six daughters of Mr. and Mrs. Mustard each...,Each daughter shares the same brother.,Some daughters get married and have their own ...,Some brothers were not loved by family and mov...,None of above.,2,[Some brothers were not loved by family and mo...,"[2, 1, 0, 3]",2,1,SR,SP-0
2,SP-0_CR,"A chess team has five players, and each player...",Each player shares the same coach.,Some coaches get a raise.,Some players are backups and not allowed to play.,None of above.,0,"[Each player shares the same coach., Some play...","[0, 2, 1, 3]",0,1,CR,SP-0
3,SP-1,A woman shoots her husband. Then she holds him...,The woman was a photographer. She shot a pictu...,The woman gets arrested for murder after dinner.,The woman gets a new partner.,None of above.,2,[The woman gets arrested for murder after dinn...,"[1, 2, 0, 3]",2,1,OG,SP-1
4,SP-1_SR,An individual shoots their spouse. She continu...,The woman was a photographer. She shot a pictu...,The woman gets arrested for murder after dinner.,The woman gets a new partner.,None of above.,1,[The woman gets arrested for murder after dinn...,"[1, 0, 2, 3]",1,1,SR,SP-1


In [25]:
# Revising the code to handle cases where no correct entries exist for some groups

# Calculate the 'Original' metric
original_correct_count = df[(df['group'] == 'OG') & (df['correct'] == 1)].shape[0]
original_total_count = df[df['group'] == 'OG'].shape[0]
original_metric = original_correct_count / original_total_count if original_total_count else 0

# Calculate the 'Semantic' metric
semantic_correct_count = df[(df['group'] == 'SR') & (df['correct'] == 1)].shape[0]
semantic_total_count = df[df['group'] == 'SR'].shape[0]
semantic_metric = semantic_correct_count / semantic_total_count if semantic_total_count else 0

# Calculate the 'Context' metric
context_correct_count = df[(df['group'] == 'CR') & (df['correct'] == 1)].shape[0]
context_total_count = df[df['group'] == 'CR'].shape[0]
context_metric = context_correct_count / context_total_count if context_total_count else 0



In [26]:
# Group by 'stem' and then calculate the combined metrics
grouped = df.groupby('stem')

# Initialize counts for the combined metrics
ori_sem_count = 0
ori_sem_con_count = 0

# Iterate over each group
for stem, group in grouped:
    # Check if all conditions for 'Ori & Sem' are met within this stem group
    if ((group['group'] == 'OG') & (group['correct'] == 1)).any() and \
       ((group['group'] == 'SR') & (group['correct'] == 1)).any():
        ori_sem_count += 1

    # Check if all conditions for 'Ori & Sem & Con' are met within this stem group
    if ((group['group'] == 'OG') & (group['correct'] == 1)).any() and \
       ((group['group'] == 'SR') & (group['correct'] == 1)).any() and \
       ((group['group'] == 'CR') & (group['correct'] == 1)).any():
        ori_sem_con_count += 1

# Calculate 'Ori & Sem' and 'Ori & Sem & Con' metrics
ori_sem_metric = ori_sem_count / grouped.ngroups if grouped.ngroups else 0
ori_sem_con_metric = ori_sem_con_count / grouped.ngroups if grouped.ngroups else 0




In [27]:
overall_correct_count = df[df['correct'] == 1].shape[0]
overall_total_count = df.shape[0]
overall_metric = overall_correct_count / overall_total_count if overall_total_count else 0

# FlanT5 (780M) metrics

In [11]:
print('original', original_metric)
print('semantic', semantic_metric)
print('context', context_metric)
print('ori_sem', ori_sem_metric)
print('ori_sem_con', ori_sem_con_metric)
print('overall', overall_metric)

original 0.17159763313609466
semantic 0.1834319526627219
context 0.24260355029585798
ori_sem 0.11242603550295859
ori_sem_con 0.04142011834319527
overall 0.1992110453648915


# Roberta-l metrics

In [28]:
print('original', original_metric)
print('semantic', semantic_metric)
print('context', context_metric)
print('ori_sem', ori_sem_metric)
print('ori_sem_con', ori_sem_con_metric)
print('overall', overall_metric)

original 0.44970414201183434
semantic 0.41420118343195267
context 0.47928994082840237
ori_sem 0.33136094674556216
ori_sem_con 0.20710059171597633
overall 0.4477317554240631
